### Установка пакетов

In [5]:
!pip install pyspark

### Импорт библиотек

In [6]:
import pandas as pd
import random
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
import urllib.request

### Генерация номера задания

In [7]:
# random.randint(1, 200)

Сгенерирован номер с заданием 99

### Создание каталога data для хранения данных и загрузка данных

In [8]:
if not os.path.exists('data'):
  os.makedirs('data')

urllib.request.urlretrieve('https://raw.githubusercontent.com/Losyash/sepbd_i_hw/refs/heads/master/lab_2/data/2.csv', 'data/2.csv')

('data/2.csv', <http.client.HTTPMessage at 0x7bb40f4623b0>)

### Создание экземпляра SparkSession

In [9]:
spark = SparkSession.builder.appName('Homework_2_lab2').getOrCreate()

### Задание 1. Считать файл 1.csv и создать Spark DataFrame

In [10]:
df = spark.read.format("csv").option("header", "true").load("data/2.csv")
df.show(5)

+-------+-----------+-------------------+-------+-------+-------+-------+
|item_id|description|           category|  price| weight| length|  width|
+-------+-----------+-------------------+-------+-------+-------+-------+
|      1|     Item 1|    Bath and Beauty|3045.71|1086.34|1324.01|  11.18|
|      2|     Item 2|  Pantry and Snacks|1500.43|  33.01|1265.36| 697.48|
|      3|     Item 3|Musical Instruments| 125.81|5849.18|2265.22|2856.18|
|      4|     Item 4|      Baby Products|1065.07|2742.43|1283.44|3842.33|
|      5|     Item 5|      Baby Products|3343.72|3108.81|1525.39|1728.18|
+-------+-----------+-------------------+-------+-------+-------+-------+
only showing top 5 rows



### Задание 2. Вывести первые 8 строк

In [11]:
df.show(8)

+-------+-----------+-------------------+-------+-------+-------+-------+
|item_id|description|           category|  price| weight| length|  width|
+-------+-----------+-------------------+-------+-------+-------+-------+
|      1|     Item 1|    Bath and Beauty|3045.71|1086.34|1324.01|  11.18|
|      2|     Item 2|  Pantry and Snacks|1500.43|  33.01|1265.36| 697.48|
|      3|     Item 3|Musical Instruments| 125.81|5849.18|2265.22|2856.18|
|      4|     Item 4|      Baby Products|1065.07|2742.43|1283.44|3842.33|
|      5|     Item 5|      Baby Products|3343.72|3108.81|1525.39|1728.18|
|      6|     Item 6|         Stationery|2889.09| 618.78|4413.94|1073.71|
|      7|     Item 7|        Accessories|2401.08|2814.12|6174.96|2981.73|
|      8|     Item 8|      Baby Products|2440.76| 4332.7|3257.63|5492.77|
+-------+-----------+-------------------+-------+-------+-------+-------+
only showing top 8 rows



In [12]:
df.head(8)

[Row(item_id='1', description='Item 1', category='Bath and Beauty', price='3045.71', weight='1086.34', length='1324.01', width='11.18'),
 Row(item_id='2', description='Item 2', category='Pantry and Snacks', price='1500.43', weight='33.01', length='1265.36', width='697.48'),
 Row(item_id='3', description='Item 3', category='Musical Instruments', price='125.81', weight='5849.18', length='2265.22', width='2856.18'),
 Row(item_id='4', description='Item 4', category='Baby Products', price='1065.07', weight='2742.43', length='1283.44', width='3842.33'),
 Row(item_id='5', description='Item 5', category='Baby Products', price='3343.72', weight='3108.81', length='1525.39', width='1728.18'),
 Row(item_id='6', description='Item 6', category='Stationery', price='2889.09', weight='618.78', length='4413.94', width='1073.71'),
 Row(item_id='7', description='Item 7', category='Accessories', price='2401.08', weight='2814.12', length='6174.96', width='2981.73'),
 Row(item_id='8', description='Item 8', c

### Задание 3. Отфильтровать данные, где ```price < 4779.0```

In [13]:
df_filtered = df.filter(df.price < 4779.0)
df_filtered.show(5)

+-------+-----------+-------------------+-------+-------+-------+-------+
|item_id|description|           category|  price| weight| length|  width|
+-------+-----------+-------------------+-------+-------+-------+-------+
|      1|     Item 1|    Bath and Beauty|3045.71|1086.34|1324.01|  11.18|
|      2|     Item 2|  Pantry and Snacks|1500.43|  33.01|1265.36| 697.48|
|      3|     Item 3|Musical Instruments| 125.81|5849.18|2265.22|2856.18|
|      4|     Item 4|      Baby Products|1065.07|2742.43|1283.44|3842.33|
|      5|     Item 5|      Baby Products|3343.72|3108.81|1525.39|1728.18|
+-------+-----------+-------------------+-------+-------+-------+-------+
only showing top 5 rows



### Задание 4. Подсчитать, сколько строк было отфильтровано на предыдущем шаге 3

In [14]:
df.count() - df_filtered.count()

120

### Задание 5. Выполнить группировку по полю category, для каждой категории рассчитав ```количество элементов в группе, min price, avg weight```

In [15]:
df_grouped = df_filtered.groupBy('category') \
   .agg({ 'item_id': 'count', 'price': 'min', 'weight': 'avg' })

df_grouped.show()

+-------------------+--------------+----------+------------------+
|           category|count(item_id)|min(price)|       avg(weight)|
+-------------------+--------------+----------+------------------+
|        Accessories|            26|    101.46|2112.1780769230763|
|       Art Supplies|            22|    116.13| 2517.932272727272|
|      Baby Products|            29|   1013.13|           3045.91|
|    Bath and Beauty|            28|   1195.44|2338.1842857142856|
| Bedding and Linens|            25|   1218.29|         3319.9236|
|              Books|            28|   1002.06| 2800.758928571428|
|           Clothing|            25|   1144.66|          2933.078|
|        Electronics|            23|   1265.69| 1867.644782608696|
|          Furniture|            17|   1064.43|1704.3170588235296|
|    Gaming Consoles|            17|   1034.54|1461.5329411764706|
|         Home Goods|            23|   1012.46|2314.7847826086954|
| Kitchen Appliances|            28|   1021.84|          2879.

### Задание 6. Записать полученный набор данных в файл r_2.json

In [16]:
df_grouped.write.json('data/r_2.json', mode='overwrite')